In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
from scipy.stats import zscore
from math import radians, cos, sin, asin, sqrt
import pydot
import seaborn as sns

In [4]:
import keras
from keras import metrics
from keras import regularizers
from keras.models import Model, Sequential, load_model
from keras.layers import Embedding, Flatten, Input, Dense, Dropout, Flatten, Activation
from keras.optimizers import Adam, RMSprop
from keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint
from keras.utils import plot_model

Using TensorFlow backend.


In [5]:
def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles
    return c * r

In [6]:
kc_raw_data = pd.read_csv('kc_house_data.csv')
list(kc_raw_data.columns)

['id',
 'date',
 'price',
 'bedrooms',
 'bathrooms',
 'sqft_living',
 'sqft_lot',
 'floors',
 'waterfront',
 'view',
 'condition',
 'grade',
 'sqft_above',
 'sqft_basement',
 'yr_built',
 'yr_renovated',
 'zipcode',
 'lat',
 'long',
 'sqft_living15',
 'sqft_lot15']

In [7]:
kc_raw_data['sale_yr'] = pd.to_numeric(kc_raw_data.date.str.slice(0,4))
kc_raw_data['sale_month'] = pd.to_numeric(kc_raw_data.date.str.slice(4,6))
kc_raw_data['sale_day'] = pd.to_numeric(kc_raw_data.date.str.slice(6,8))
kc_raw_data['house_age'] = kc_raw_data['sale_yr'] - kc_raw_data['yr_built']
FIXED_LONG = -122.213896
FIXED_LAT = 47.560053
kc_raw_data['distance'] = kc_raw_data.apply(lambda row: haversine(FIXED_LONG, FIXED_LAT, row['long'], row['lat']), axis=1)
kc_raw_data['greater_long'] = (kc_raw_data['long'] >= FIXED_LONG).astype(int)
kc_raw_data['less_long'] = (kc_raw_data['long'] < FIXED_LONG).astype(int)
kc_raw_data['greater_lat'] = (kc_raw_data['lat'] >= FIXED_LAT).astype(int)
kc_raw_data['less_lat'] = (kc_raw_data['lat'] < FIXED_LAT).astype(int)

kc_data = pd.DataFrame(kc_raw_data, columns=[
        'sale_yr','sale_month','sale_day','house_age', 'distance', 'greater_long', 'less_long',
        'greater_lat','less_lat', 'view', 'waterfront',
        'bedrooms','bathrooms','sqft_living','sqft_lot','floors',
        'condition','grade','sqft_above','sqft_basement','yr_built','yr_renovated',
        'zipcode','sqft_living15','sqft_lot15','price'])

label_column = 'price'

#embedding code change
zipcode_constant = 98000
kc_data['zipcode'] = kc_data['zipcode'] - zipcode_constant
kc_data = kc_data.sample(frac=1)
kc_data.describe()


,sale_yr,sale_month,sale_day,house_age,distance,greater_long,less_long,greater_lat,less_lat,view,...,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,sqft_living15,sqft_lot15,price
count,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,...,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,2.161300e+04
mean,2014.322954,6.574423,15.688197,43.317818,16.918985,0.463517,0.536483,0.536251,0.463749,0.234303,...,3.409430,7.656873,1788.390691,291.509045,1971.005136,84.402258,77.939805,1986.552492,12768.455652,5.400881e+05
std,0.467616,3.115308,8.635063,29.375493,7.925265,0.498679,0.498679,0.498696,0.498696,0.766318,...,0.650743,1.175459,828.090978,442.575043,29.373411,401.679240,53.505026,685.391304,27304.179631,3.671272e+05
min,2014.000000,1.000000,1.000000,-1.000000,0.171161,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.000000,1.000000,290.000000,0.000000,1900.000000,0.000000,1.000000,399.000000,651.000000,7.500000e+04
25%,2014.000000,4.000000,8.000000,18.000000,11.553287,0.000000,0.000000,0.000000,0.000000,0.000000,...,3.000000,7.000000,1190.000000,0.000000,1951.000000,0.000000,33.000000,1490.000000,5100.000000,3.219500e+05
50%,2014.000000,6.000000,16.000000,40.000000,16.229901,0.000000,1.000000,1.000000,0.000000,0.000000,...,3.000000,7.000000,1560.000000,0.000000,1975.000000,0.000000,65.000000,1840.000000,7620.000000,4.500000e+05
75%,2015.000000,9.000000,23.000000,63.000000,21.973207,1.000000,1.000000,1.000000,1.000000,0.000000,...,4.000000,8.000000,2210.000000,560.000000,1997.000000,0.000000,118.000000,2360.000000,10083.000000,6.450000e+05
max,2015.000000,12.000000,31.000000,115.000000,69.464844,1.000000,1.000000,1.000000,1.000000,4.000000,...,5.000000,13.000000,9410.000000,4820.000000,2015.000000,2015.000000,199.000000,6210.000000,871200.000000,7.700000e+06


In [8]:
train = kc_data.sample(frac=0.8)
'Train:' + str(train.shape)
validate = kc_data.sample(frac=0.1)
'Validate:' + str(validate.shape)
test = kc_data.sample(frac=0.1)
'Test:' + str(test.shape)

cols = list(kc_data.columns)
cols.remove(label_column)
cols.remove('zipcode')

train_x = train[cols]
train_y = train[label_column]
train_zipcode_x = train['zipcode']
train_x_mean = train_x[cols].mean(axis=0)
train_x_std = train_x[cols].std(axis=0)

validate_x = validate[cols]
validate_y = validate[label_column]
validate_zipcode_x = validate['zipcode']

test_x = test[cols]
test_y = test[label_column]
test_zipcode_x = test['zipcode']

# zscores
train_x[cols] = (train_x[cols] - train_x_mean) / train_x_std
validate_x[cols] = (validate_x[cols] - train_x_mean) / train_x_std
test_x[cols] = (test_x[cols] - train_x_mean) / train_x_std

'Train:(17290, 26)'

'Validate:(2161, 26)'

'Test:(2161, 26)'

/home/amit.dingare/.conda/envs/mlpython/lib/python3.6/site-packages/pandas/core/frame.py:3140: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


In [9]:
def model3(train_x, train_zipcode_t, train_y):
    data_zipcode = train_zipcode_x
    data_main = train_x
    data_y = train_y
    
    # There are two parts. catergorical embeddings for zipcodes and main layers for other numerical features

    zipcode_input = Input(shape=(1,), dtype='int32', name='zipcode_input')
    x = Embedding(output_dim=5, input_dim=200, input_length=1)(zipcode_input)
    zipcode_out = Flatten()(x)
    zipcode_output = Dense(1, activation='relu', name='zipcode_model_out')(zipcode_out)

    
    main_input = Input(shape=(data_main.shape[1],), name='main_input')
    lyr = keras.layers.concatenate([main_input, zipcode_out])
    lyr = Dense(100, activation="relu")(lyr)
    lyr = Dense(50, activation="relu")(lyr)
    main_output = Dense(1, name='main_output')(lyr)

    t_model = Model(
        inputs=[main_input, zipcode_input], 
        outputs=[main_output, zipcode_output]
    )
    t_model.compile(
        loss="mean_squared_error",
        optimizer=Adam(lr=0.001),
        metrics=[metrics.mae],
        loss_weights=[1.0, 0.5]
    )
    return t_model


In [10]:
keras_callbacks = [
    # ModelCheckpoint('/tmp/keras_checkpoints/model.{epoch:02d}-{val_loss:.2f}.hdf5', monitor='val_loss', save_best_only=True, verbose=2)
    # ModelCheckpoint('/tmp/keras_checkpoints/model.{epoch:02d}.hdf5', monitor='val_loss', save_best_only=True, verbose=0)
    # TensorBoard(log_dir='/tmp/keras_logs/model_3', histogram_freq=0, write_graph=True, write_images=True, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None),
    # EarlyStopping(monitor='val_mean_absolute_error', patience=20, verbose=0)
]
model = model3(train_x, train_zipcode_x, train_y)
print(model.summary())
plot_model(model, to_file='kc_housing.png', show_shapes=True, show_layer_names=True)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
zipcode_input (InputLayer)      (None, 1)            0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 1, 5)         1000        zipcode_input[0][0]              
__________________________________________________________________________________________________
main_input (InputLayer)         (None, 24)           0                                            
__________________________________________________________________________________________________
flatten_1 (Flatten)             (None, 5)            0           embedding_1[0][0]                
__________________________________________________________________________________________________
concatenat

In [11]:
epochs = 500
batch = 128

cols = list(train.columns)
cols.remove(label_column)
history = model.fit(
    [train_x, train_zipcode_x], [train_y, train_y],
    batch_size=batch,
    epochs=epochs,
    shuffle=True,
    verbose=1,
    validation_data=([validate_x, validate_zipcode_x],[validate_y, validate_y]),
    callbacks=keras_callbacks
)

Train on 17290 samples, validate on 2161 samples
Epoch 1/500
17290/17290 [==============================] - 2s 122us/step - loss: 633173239091.3777 - main_output_loss: 422097993748.8472 - zipcode_model_out_loss: 422150490423.5234 - main_output_mean_absolute_error: 540675.4276 - zipcode_model_out_mean_absolute_error: 540709.4802 - val_loss: 673059605510.1602 - val_main_output_loss: 448628619007.1708 - val_zipcode_model_out_loss: 448861973460.8792 - val_main_output_mean_absolute_error: 546058.6882 - val_zipcode_model_out_mean_absolute_error: 546203.3521
Epoch 2/500
17290/17290 [==============================] - 1s 76us/step - loss: 632172063434.2533 - main_output_loss: 421097414389.7244 - zipcode_model_out_loss: 422149299059.3999 - main_output_mean_absolute_error: 540040.9489 - zipcode_model_out_mean_absolute_error: 540708.3134 - val_loss: 670613415476.3610 - val_main_output_loss: 446183217118.8302 - val_zipcode_model_out_loss: 448860396487.6112 - val_main_output_mean_absolute_error: 544

17290/17290 [==============================] - 1s 56us/step - loss: 245043857332.9027 - main_output_loss: 33990331376.8976 - zipcode_model_out_loss: 422107051980.1189 - main_output_mean_absolute_error: 107883.3841 - zipcode_model_out_mean_absolute_error: 540670.7482 - val_loss: 266055246783.0819 - val_main_output_loss: 41646447283.8279 - val_zipcode_model_out_loss: 448817597385.5067 - val_main_output_mean_absolute_error: 110076.2713 - val_zipcode_model_out_mean_absolute_error: 546164.5104
Epoch 18/500
17290/17290 [==============================] - 1s 81us/step - loss: 244015271100.6908 - main_output_loss: 32962490596.6084 - zipcode_model_out_loss: 422105559459.3129 - main_output_mean_absolute_error: 106372.0822 - zipcode_model_out_mean_absolute_error: 540669.5014 - val_loss: 264802031787.5354 - val_main_output_loss: 40394008855.5743 - val_zipcode_model_out_loss: 448816041989.6863 - val_main_output_mean_absolute_error: 108960.6554 - val_zipcode_model_out_mean_absolute_error: 546163.2127

17290/17290 [==============================] - 1s 55us/step - loss: 237799123233.7286 - main_output_loss: 26757156568.8227 - zipcode_model_out_loss: 422083934643.1852 - main_output_mean_absolute_error: 98778.3871 - zipcode_model_out_mean_absolute_error: 540651.0587 - val_loss: 256790500114.5988 - val_main_output_loss: 32393666446.2749 - val_zipcode_model_out_loss: 448793667950.7635 - val_main_output_mean_absolute_error: 99969.3608 - val_zipcode_model_out_mean_absolute_error: 546144.5709
Epoch 34/500
17290/17290 [==============================] - 1s 49us/step - loss: 237601048701.7939 - main_output_loss: 26559796939.3194 - zipcode_model_out_loss: 422082503919.1505 - main_output_mean_absolute_error: 98609.0893 - zipcode_model_out_mean_absolute_error: 540649.8397 - val_loss: 256549839405.2531 - val_main_output_loss: 32153734410.7802 - val_zipcode_model_out_loss: 448792210631.4928 - val_main_output_mean_absolute_error: 99493.8943 - val_zipcode_model_out_mean_absolute_error: 546143.3963
Epo

17290/17290 [==============================] - 1s 59us/step - loss: 235344065930.0831 - main_output_loss: 24313316194.3431 - zipcode_model_out_loss: 422061500328.5839 - main_output_mean_absolute_error: 95002.0341 - zipcode_model_out_mean_absolute_error: 540631.9376 - val_loss: 253753601390.2896 - val_main_output_loss: 29368275599.8149 - val_zipcode_model_out_loss: 448770656611.3910 - val_main_output_mean_absolute_error: 95554.0391 - val_zipcode_model_out_mean_absolute_error: 546125.3496
Epoch 50/500
17290/17290 [==============================] - 1s 53us/step - loss: 235225932399.5206 - main_output_loss: 24195853865.3983 - zipcode_model_out_loss: 422060158072.4636 - main_output_mean_absolute_error: 94689.4945 - zipcode_model_out_mean_absolute_error: 540630.7982 - val_loss: 253588025626.4174 - val_main_output_loss: 29203403060.0056 - val_zipcode_model_out_loss: 448769245646.4822 - val_main_output_mean_absolute_error: 95299.0894 - val_zipcode_model_out_mean_absolute_error: 546124.2065
Epo

17290/17290 [==============================] - 1s 82us/step - loss: 233513189957.8263 - main_output_loss: 22493457795.6868 - zipcode_model_out_loss: 422039464506.2181 - main_output_mean_absolute_error: 90732.1128 - zipcode_model_out_mean_absolute_error: 540613.4421 - val_loss: 251480758837.7825 - val_main_output_loss: 27106818769.2068 - val_zipcode_model_out_loss: 448747878196.2425 - val_main_output_mean_absolute_error: 91754.4317 - val_zipcode_model_out_mean_absolute_error: 546106.6591
Epoch 66/500
17290/17290 [==============================] - 1s 64us/step - loss: 233415582491.0362 - main_output_loss: 22396539267.8644 - zipcode_model_out_loss: 422038085077.6542 - main_output_mean_absolute_error: 90808.5962 - zipcode_model_out_mean_absolute_error: 540612.3357 - val_loss: 251447576552.3072 - val_main_output_loss: 27074291504.4516 - val_zipcode_model_out_loss: 448746569189.7010 - val_main_output_mean_absolute_error: 91105.8617 - val_zipcode_model_out_mean_absolute_error: 546105.5915
Epo

...

17290/17290 [==============================] - 2s 88us/step - loss: 222836377083.3212 - main_output_loss: 12126052887.6308 - zipcode_model_out_loss: 421420649650.2672 - main_output_mean_absolute_error: 67316.5205 - zipcode_model_out_mean_absolute_error: 540077.2163 - val_loss: 238680129308.5497 - val_main_output_loss: 14620422143.7631 - val_zipcode_model_out_loss: 448119414219.1652 - val_main_output_mean_absolute_error: 68523.6676 - val_zipcode_model_out_mean_absolute_error: 545569.8758
Epoch 434/500
17290/17290 [==============================] - 1s 70us/step - loss: 222829739907.8644 - main_output_loss: 12120520748.0042 - zipcode_model_out_loss: 421418438536.1286 - main_output_mean_absolute_error: 67275.8690 - zipcode_model_out_mean_absolute_error: 540075.2638 - val_loss: 238675962776.2258 - val_main_output_loss: 14617182681.3808 - val_zipcode_model_out_loss: 448117561060.6349 - val_main_output_mean_absolute_error: 68466.1410 - val_zipcode_model_out_mean_absolute_error: 545568.3008
Ep

17290/17290 [==============================] - 1s 55us/step - loss: 222739154764.6667 - main_output_loss: 12044962899.0927 - zipcode_model_out_loss: 421388383612.8167 - main_output_mean_absolute_error: 67099.5075 - zipcode_model_out_mean_absolute_error: 540048.3627 - val_loss: 238571694454.3452 - val_main_output_loss: 14527992256.5035 - val_zipcode_model_out_loss: 448087404477.6603 - val_main_output_mean_absolute_error: 68377.9195 - val_zipcode_model_out_mean_absolute_error: 545541.8380
Epoch 450/500
17290/17290 [==============================] - 1s 55us/step - loss: 222731238313.4131 - main_output_loss: 12037852267.1972 - zipcode_model_out_loss: 421386771552.2998 - main_output_mean_absolute_error: 67130.3228 - zipcode_model_out_mean_absolute_error: 540047.0251 - val_loss: 238582115178.7358 - val_main_output_loss: 14539481017.8695 - val_zipcode_model_out_loss: 448085270448.3924 - val_main_output_mean_absolute_error: 68334.1526 - val_zipcode_model_out_mean_absolute_error: 545539.8865
Ep

17290/17290 [==============================] - 1s 75us/step - loss: 222661064365.9438 - main_output_loss: 11982879532.0930 - zipcode_model_out_loss: 421356369223.7511 - main_output_mean_absolute_error: 67041.0177 - zipcode_model_out_mean_absolute_error: 540019.7472 - val_loss: 238496462234.8320 - val_main_output_loss: 14469140292.3535 - val_zipcode_model_out_loss: 448054639934.9042 - val_main_output_mean_absolute_error: 68304.7351 - val_zipcode_model_out_mean_absolute_error: 545512.9322
Epoch 466/500
17290/17290 [==============================] - 1s 64us/step - loss: 222656287871.8075 - main_output_loss: 11979116072.5691 - zipcode_model_out_loss: 421354343388.1097 - main_output_mean_absolute_error: 66966.7815 - zipcode_model_out_mean_absolute_error: 540018.0068 - val_loss: 238504975129.2328 - val_main_output_loss: 14478805994.4396 - val_zipcode_model_out_loss: 448052334302.0009 - val_main_output_mean_absolute_error: 68226.9974 - val_zipcode_model_out_mean_absolute_error: 545510.8664
Ep

17290/17290 [==============================] - 1s 70us/step - loss: 222578983480.5598 - main_output_loss: 11916958414.9913 - zipcode_model_out_loss: 421324049377.7953 - main_output_mean_absolute_error: 66877.3831 - zipcode_model_out_mean_absolute_error: 539990.7150 - val_loss: 238424224777.0032 - val_main_output_loss: 14413416755.0578 - val_zipcode_model_out_loss: 448021617584.8663 - val_main_output_mean_absolute_error: 68208.3337 - val_zipcode_model_out_mean_absolute_error: 545483.5725
Epoch 482/500
17290/17290 [==============================] - 1s 75us/step - loss: 222578770738.7262 - main_output_loss: 11917790193.4307 - zipcode_model_out_loss: 421321963657.6389 - main_output_mean_absolute_error: 66912.6002 - zipcode_model_out_mean_absolute_error: 539988.9594 - val_loss: 238419005081.7658 - val_main_output_loss: 14409168177.3994 - val_zipcode_model_out_loss: 448019670686.5044 - val_main_output_mean_absolute_error: 68113.3816 - val_zipcode_model_out_mean_absolute_error: 545481.8559
Ep

17290/17290 [==============================] - 2s 96us/step - loss: 222508971201.1919 - main_output_loss: 11863303664.9865 - zipcode_model_out_loss: 421291334723.8718 - main_output_mean_absolute_error: 66802.8604 - zipcode_model_out_mean_absolute_error: 539961.5578 - val_loss: 238348371269.5382 - val_main_output_loss: 14353998129.1624 - val_zipcode_model_out_loss: 447988743547.5576 - val_main_output_mean_absolute_error: 67931.9599 - val_zipcode_model_out_mean_absolute_error: 545454.6773
Epoch 498/500
17290/17290 [==============================] - 1s 75us/step - loss: 222511215672.7375 - main_output_loss: 11866567634.4264 - zipcode_model_out_loss: 421289297005.3293 - main_output_mean_absolute_error: 66766.4674 - zipcode_model_out_mean_absolute_error: 539959.6143 - val_loss: 238344006438.5007 - val_main_output_loss: 14350785332.7163 - val_zipcode_model_out_loss: 447986438718.3119 - val_main_output_mean_absolute_error: 67955.0159 - val_zipcode_model_out_mean_absolute_error: 545452.5704
Ep

In [12]:
score = model.evaluate([test_x, test_zipcode_x], [test_y, test_y], verbose=0)

In [13]:
print('score:{}'.format(score))

score:[215222688681.28458, 11582847233.776955, 407279682559.5261, 64999.81490773947, 530025.2308682323]


In [ ]:
from sklearn.manifold import TSNE
import seaborn as sns

zipcode_embeddings = model.layers[1].get_weights()[0]
labels = train_zipcode_x
zipcode_embeddings.shape
tsne_model = TSNE(perplexity=200, n_components=2, init='pca', n_iter=2500, random_state=23)
new_values = tsne_model.fit_transform(zipcode_embeddings)

x1 = []
y1 = []
avg_price1 = []
x2 = []
y2 = []
avg_price2 = []
for index, value in enumerate(train_zipcode_x):
    zipcode = train_zipcode_x.iloc[index]
    price = train_y.iloc[index]
    if price > 0:
        avg_price1.append(price)
        x1.append(new_values[zipcode][0])
        y1.append(new_values[zipcode][1])
    else:
        avg_price2.append(price)
        x2.append(new_values[zipcode][0])
        y2.append(new_values[zipcode][1])

f, ax = plt.subplots(1, 1)

#cmap = sns.choose_colorbrewer_palette('s', as_cmap=True)
cmap = sns.cubehelix_palette(n_colors=10, start=0.3, rot=0.4, gamma=1.0, hue=1.0, light=0.9, dark=0.1, as_cmap=True)
axs0 = ax.scatter(x1, y1, s=20, c=avg_price1, cmap=cmap)
f.colorbar(axs0, ax=ax, orientation='vertical')

#cmap = sns.choose_colorbrewer_palette('s', as_cmap=True)
#cmap = sns.cubehelix_palette('s', as_cmap=True)
#axs1 = ax[1].scatter(x2, y2, s=8, c=avg_price2, cmap=cmap)
#f.colorbar(axs1, ax=ax[1], orientation='vertical')
f   

(200, 5)